In [1]:
from os import path

PROJECT_ROOT = path.abspath(path.join(globals()['_dh'][0], '..'))
DATALAKE_PATH = path.abspath(path.join(PROJECT_ROOT, '..', '..', 'datalake', 'mlpbertproj'))


In [2]:
import base64
import os

import pandas as pd


def encode_text_body(text_body: str | bytes) -> str:
    """
    Returns Base64-encoded string version of a UTF-8 string or bytes representing a UTF-8 string
    """
    if isinstance(text_body, bytes):
        return base64.b64encode(text_body).decode('utf-8')
    else:
        return base64.b64encode(text_body.encode('utf-8')).decode('utf-8')


def text_as_base64(fqfn: str) -> tuple[str, str]:
    with open(fqfn, 'r') as f:
        text_bytes = f.read()
        b64str = encode_text_body(text_bytes)
        return text_bytes, b64str


def read(folder_path: str, labels: list[str] = ['0', '1'], k: int = 3) -> pd.DataFrame:
    txt_labels: list[int] = []
    txt_fqfns: list[str] = []
    txt_file_names: list[str] = []
    txt_lengths: list[int] = []
    txt_b64s: list[str] = []
    txt_contents: list[str] = []

    for label in labels:
        label_counter = 0
        label_folder = os.path.join(folder_path, label)
        if not os.path.isdir(label_folder):
            continue  # Skip if the folder does not exist

        for file_name in os.listdir(label_folder):
            if file_name.endswith('.ps1'):
                label_counter += 1
                if label_counter > k:
                    break

                txt_file_names.append(file_name)

                text_fqfn = os.path.join(label_folder, file_name)
                txt_fqfns.append(text_fqfn)

                text_bytes, b64str = text_as_base64(text_fqfn)
                txt_b64s.append(b64str)
                txt_contents.append(text_bytes)

                txt_lengths.append(len(text_bytes))
                txt_labels.append(int(label))


    return pd.DataFrame(data={
        'file_name': txt_file_names,
        'fqfn': txt_fqfns,
        'text': txt_contents,
        'text_size': txt_lengths,
        'text_body_b64': txt_b64s,
        'label': txt_labels,
    })


In [3]:
df = read(DATALAKE_PATH, labels=['0', '1'], k=3)
df[['text', 'text_size', 'text_body_b64', 'label']]

,text,text_size,text_body_b64,label
0,\nfunction Test-CPrivilege\n{\n \n [Cmdl...,542,CmZ1bmN0aW9uIFRlc3QtQ1ByaXZpbGVnZQp7CiAgICAKIC...,0
1,﻿\n\n\n\n\nfunction Get-AuthToken {\n\n\n\n[cm...,87178,77u/CgoKCgpmdW5jdGlvbiBHZXQtQXV0aFRva2VuIHsKCg...,0
2,\n\n\n\n\n\n\n\n\n\n\n\n$CategoryName = 'Carbo...,6819,CgoKCgoKCgoKCgoKJENhdGVnb3J5TmFtZSA9ICdDYXJib2...,0
3,"\n$C3H = '[DllImport(""kernel32.dll"")]public st...",2337,CiRDM0ggPSAnW0RsbEltcG9ydCgia2VybmVsMzIuZGxsIi...,1
4,\n(New-Object System.Net.WebClient).DownloadFi...,157,CihOZXctT2JqZWN0IFN5c3RlbS5OZXQuV2ViQ2xpZW50KS...,1
5,"\n$c = '[DllImport(""kernel32.dll"")]public stat...",2324,CiRjID0gJ1tEbGxJbXBvcnQoImtlcm5lbDMyLmRsbCIpXX...,1


In [4]:
from mlpbertproj.classifier.inf_server_driver import InferenceServerDriver

model = InferenceServerDriver()
df['outcome'] = model.predict(inputs=df)
print(df[['text', 'text_size', 'text_body_b64', 'label', 'outcome']])


2025-02-07 14:03:44,293 - tensorcraft - INFO - XLA Device Not Supported: No module named 'torch_xla'
2025-02-07 14:03:44,301 - tensorcraft - INFO - Pytorch version=2.6.0 preferred device=mps build with MPS support=True
2025-02-07 14:03:44,303 - tensorcraft - INFO - XLA Device Not Supported: No module named 'torch_xla'
2025-02-07 14:03:44,303 - tensorcraft - INFO - Reuse GPU Tensor: False
2025-02-07 14:03:44,304 - tensorcraft - INFO - resolved device_name: mps compute_device: mps tensor_device: mps
2025-02-07 14:03:44,304 - tensorcraft - INFO - LM Components are being loaded from microsoft/graphcodebert-base...
Some weights of RobertaModel were not initialized from the model checkpoint at microsoft/graphcodebert-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
2025-02-07 14:03:44,915 - tensorcraft - INFO - LM Components successfully l

Label=[0]
Label=[0]
Label=[0]
Label=[1]
Label=[1]
Label=[1]
                                                text  text_size  \
0  \nfunction Test-CPrivilege\n{\n    \n    [Cmdl...        542   
1  ﻿\n\n\n\n\nfunction Get-AuthToken {\n\n\n\n[cm...      87178   
2  \n\n\n\n\n\n\n\n\n\n\n\n$CategoryName = 'Carbo...       6819   
3  \n$C3H = '[DllImport("kernel32.dll")]public st...       2337   
4  \n(New-Object System.Net.WebClient).DownloadFi...        157   
5  \n$c = '[DllImport("kernel32.dll")]public stat...       2324   

                                       text_body_b64  label  outcome  
0  CmZ1bmN0aW9uIFRlc3QtQ1ByaXZpbGVnZQp7CiAgICAKIC...      0        0  
1  77u/CgoKCgpmdW5jdGlvbiBHZXQtQXV0aFRva2VuIHsKCg...      0        0  
2  CgoKCgoKCgoKCgoKJENhdGVnb3J5TmFtZSA9ICdDYXJib2...      0        0  
3  CiRDM0ggPSAnW0RsbEltcG9ydCgia2VybmVsMzIuZGxsIi...      1        1  
4  CihOZXctT2JqZWN0IFN5c3RlbS5OZXQuV2ViQ2xpZW50KS...      1        1  
5  CiRjID0gJ1tEbGxJbXBvcnQoImtlcm5lbDMyLmRsb